In [ ]:
! pwd

In [ ]:
! head -n 5 curso-apache-spark-platzi/files/deportistaError.csv

In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType,StringType,FloatType

In [ ]:
spark = SparkSession.builder.appName("UDFs").master("local").getOrCreate()

In [ ]:
path = "/home/gerardo/proyectos_PLATZI/Curso_Spark/curso-apache-spark-platzi/files/"
deportista_error_schema = StructType([
 StructField("deportista_id",IntegerType(),False),
    StructField("nombre",StringType(),False),
    StructField("genero",IntegerType(),False),
    StructField("edad",IntegerType(),False),
    StructField("altura",IntegerType(),False),
    StructField("peso",FloatType(),False),
    StructField("equipo_id",IntegerType(),False)
])
deportista_error_df = spark.read.schema(deportista_error_schema) \
                .csv(path + "deportistaError.csv",header=True)

In [ ]:
deportista_error_df.show(5)

In [ ]:
from pyspark.sql.functions import udf
#CREO MI FUNCIÓN PERSONALIZADA
def select_gen(s):
    return "MALE" if s == 1 else "FEMALE"

In [ ]:
#LA VUELVO UN UDF PARA USARLA CON DATAFRAMES Y LA REGISTRO PARA USARLA COMO SQL
select_gen_udf = udf(select_gen,StringType())
spark.udf.register("select_gen_udf",select_gen_udf)

In [ ]:
deportista_error_df.select("nombre",select_gen_udf("genero").alias("Genero")).show(5)

In [ ]:
deportista_error_df.createOrReplaceTempView("deportista_error")

In [ ]:
spark.sql("SELECT nombre,select_gen_udf(genero) AS GENERO FROM deportista_error").show(5)